#### We finetune this LM : roberta-base-squad2 on dataset.txt having question and answer

In [4]:
from transformers import RobertaForQuestionAnswering, RobertaTokenizer, RobertaTokenizerFast
import pickle
import random
import torch
from torch.utils.data import Dataset

In [5]:
dataset_file = '../data/LM_dataset.txt'
graph_file = '../models/graph.pkl'

In [6]:
# Get data from LM Dataset.txt
questions = []
answers = []
with open(dataset_file, 'r') as file:
    lines = file.readlines()
    for i, line in enumerate(lines):
        if i % 3 == 0:
            questions.append(line.strip())
        elif i % 3 == 1:
            answers.append(line.strip()[8:])
        else:
            continue

In [7]:
# Read graph for context
with open(graph_file, 'rb') as f:
    graph = pickle.load(f)

In [8]:
# Helper function to create context 
def get_context_from_qa(graph, ques, ans):
    ques = ques.lower()
    ans = ans.lower()
    context = [ans]
    for node in graph.nodes():
        if node in ques and node is not None:
            context.append(node)
    random.shuffle(context)
    return ' '.join(context)

In [9]:
# Create dataset in standard QA format
dataset = []
for i in range(len(questions)):
    sample = {}
    sample['question'] = str(questions[i])
    sample['context'] = get_context_from_qa(graph, questions[i], answers[i])
    text = str(answers[i].lower())
    answer= {}
    answer['text'] = [text]
    answer['answer_start'] = [sample['context'].find(text)] 
    sample['answer'] = answer 
    dataset.append(sample)

In [10]:
#dataset

In [11]:
# Generate custom dataset with tokenizers to get tensors
class CustomQADataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context = self.data[idx]['context']
        question = self.data[idx]['question']
        answer = self.data[idx]['answer']

        inputs = self.tokenizer.encode_plus(
            question,
            context,
            add_special_tokens=True,
            return_tensors='pt',
            max_length=128,
            truncation=True,
            padding='max_length',
            return_offsets_mapping=True
        )

        start_positions = answer['answer_start'][0]
        end_positions = answer['answer_start'][0] + len(answer['text'][0])

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'start_positions': start_positions,
            'end_positions': end_positions
        }

In [12]:
tokenizer = RobertaTokenizerFast.from_pretrained('deepset/roberta-base-squad2')

In [13]:
final_custom_dataset = CustomQADataset(dataset, tokenizer)

### Dataset is ready .. lets begin finetuning LM

In [25]:
from torch.utils.data import DataLoader
import tqdm

In [16]:
model = RobertaForQuestionAnswering.from_pretrained('deepset/roberta-base-squad2')

In [22]:
N = len(final_custom_dataset)

In [23]:
train_loader = DataLoader(final_custom_dataset, batch_size=16, shuffle=True)

In [31]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001)
criterion = torch.nn.MSELoss()

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    batchcount = 0
    epoch_loss = 0
    for batch in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        inputs = {key: batch[key].to(model.device) for key in batch}
        outputs = model(**inputs)
        start_pred = torch.tensor(torch.argmax(outputs.start_logits, dim = 1).to(torch.float), requires_grad=True)
        #start_pred = torch.argmax(outputs.start_logits, dim = 1).clone().detach().to(torch.float)
        end_pred = torch.tensor(torch.argmax(outputs.end_logits, dim = 1).to(torch.float), requires_grad=True)
        #start_pred = torch.argmax(outputs.end_logits, dim = 1).clone().detach().to(torch.float)
        start_loss = criterion(start_pred, batch['start_positions'].to(torch.float))
        end_loss = criterion(end_pred, batch['end_positions'].to(torch.float))
        loss = start_loss + end_loss
        loss.backward(retain_graph=True)
        optimizer.step()
        batchcount = batchcount + 1
        #print('Loss: ' + str(loss) + ' for batch ' + str(batchcount))
        epoch_loss = epoch_loss + loss
    print('+++++++++++++++++ Epoch' + str(epoch + 1) + ' Loss: ' + str(epoch_loss / N) + ' +++++++++++++++')
    # Save finetuned model
    model.save_pretrained('../models/fine_tuned_roberta_squad2')
    tokenizer.save_pretrained('../models/fine_tuned_roberta_squad2')

  0%|                                                                                                                                                                               | 0/42 [00:00<?, ?it/s]/tmp/ipykernel_68823/233285263.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  start_pred = torch.tensor(torch.argmax(outputs.start_logits, dim = 1).to(torch.float), requires_grad=True)
/tmp/ipykernel_68823/233285263.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  end_pred = torch.tensor(torch.argmax(outputs.end_logits, dim = 1).to(torch.float), requires_grad=True)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████

+++++++++++++++++ Epoch1 Loss: tensor(117.9385, grad_fn=<DivBackward0>) +++++++++++++++


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [01:04<00:00,  1.53s/it]


+++++++++++++++++ Epoch2 Loss: tensor(115.3727, grad_fn=<DivBackward0>) +++++++++++++++


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [01:04<00:00,  1.54s/it]


...

+++++++++++++++++ Epoch98 Loss: tensor(117.1048, grad_fn=<DivBackward0>) +++++++++++++++


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [01:05<00:00,  1.56s/it]


+++++++++++++++++ Epoch99 Loss: tensor(112.6106, grad_fn=<DivBackward0>) +++++++++++++++


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [01:06<00:00,  1.58s/it]


+++++++++++++++++ Epoch100 Loss: tensor(117.9392, grad_fn=<DivBackward0>) +++++++++++++++
